This is an example of how to use the codes in this repo. 

In [1]:
import brainscore
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

/home/billbrod/miniconda3/envs/manifolds/lib/python3.9/site-packages/brainscore/metrics/__init__.py:37: FutureWarning: xarray subclass Score should explicitly define __slots__
  class Score(DataAssembly):


## Load Neural Data from Brainscore

For this example, we will use the MajajHong2015 data with neural data from IT and V4. 

In [2]:
# loading Neural Data
Majaj2015 = brainscore.get_assembly(name="dicarlo.MajajHong2015.public")
stimulus_set = Majaj2015.attrs["stimulus_set"].sort_values(by=['category_name', 'object_name', 'image_id'])

# and load images from paths

neural_data = Majaj2015.multi_groupby(['category_name', 'object_name', 'image_id']).mean(dim="presentation").squeeze('time_bin')
assert all(neural_data['image_id'].values == stimulus_set['image_id'].values)
# add 'variation' along 'presentation' axis and append as a multi-index
neural_data['variation'] = (('presentation'), stimulus_set['variation'])
neural_data = neural_data.set_index(presentation=['variation'], append=True)

obj_class = pd.unique(stimulus_set['object_name'])
category = pd.unique(stimulus_set["category_name"])
it_data = neural_data[neural_data['region']=="IT"]
vfour_data = neural_data[neural_data['region']=="V4"]
assert all(it_data["image_id"] == stimulus_set["image_id"])

Loading catalog from entrypoints
Loading lookup from /home/billbrod/miniconda3/envs/manifolds/lib/python3.9/site-packages/brainscore/lookup.csv


brai
brai
brai


We create a dictionary to contain the neural data. Each key of the dictionary is the layer name. The neural data in each layer is structured as a list of manifolds (classes), each with the shape (Number of Neurons/Features, Number of Examples). 

In [3]:
# create neural manifold data classifying by 'object_name'

neural_dict = {}

it_manifold_data = []
for c in obj_class:
    #print(c)
    ind = stimulus_set[stimulus_set['object_name']==c].index
    #print(neural_data['image_id'][ind].shape)
    data = np.array(it_data[:,ind].values)
    it_manifold_data.append(data)

neural_dict["IT"] = it_manifold_data

v4_manifold_data = []
for c in obj_class:
    #print(c)
    ind = stimulus_set[stimulus_set['object_name']==c].index
    #print(neural_data['image_id'][ind].shape)
    data = np.array(vfour_data[:,ind].values)
    v4_manifold_data.append(data)

neural_dict["V4"] = v4_manifold_data

for layer in neural_dict.keys():
    print(f"In {layer}, there are {len(neural_dict[layer])} manifolds/classes.")
    print(f"Each manifold has shape {neural_dict[layer][0].shape}")

In IT, there are 64 manifolds/classes.
Each manifold has shape (168, 50)
In V4, there are 64 manifolds/classes.
Each manifold has shape (88, 50)


You can save the dictionary as a json file as follow.

In [5]:
import json
from json_numpy import default,object_hook

json_object = json.dumps(neural_dict,default=default)
with open('Majaj2015object.json', 'w') as outfile:
    outfile.write(json_object)
    outfile.close()

## Calculating Capacity

Manifold analysis calculates the theoretical capacity of the layers in question along with the manifold radius and dimension. 

In [ ]:
from mftma.manifold_analysis_correlation import manifold_analysis_corr

for layer, X in neural_dict.items():
    alpha, rad, dim, _, _ = manifold_analysis_corr(X, 0, 300) 
    # Compute the mean values
    a = 1/np.mean(1/alpha)
    r = np.mean(rad)
    d = np.mean(dim)
    print(f"{layer} capacity: {a}, radius {r}, dimension {d}")